In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_ny.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1","Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1","Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
fips_check

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,605841,1009208907,4/24/21,202,PPP,THE CENTER FOR DISCOVERY INC.,641 Old Route 17,Monticello,NY,12701.0,...,999 Other,6232 Residential Intellectual and Developmenta...,6232.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"641 Old Route 17, Monticello, NY",41.701466,-74.735090,3.610595e+14
1,605842,1029567410,5/3/20,202,PPP,"BEDDING ACQUISITION, LLC",60 EAST 42ND ST SUITE 1250,NEW YORK,NY,10165.0,...,314 Textile Products,3149 Other Textile Product Mills,3149.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"60 EAST 42ND ST SUITE 1250, NEW YORK, NY",40.752097,-73.978733,NaN
2,605843,1040387100,4/9/20,202,PPP,P A L ENVIRONMENTAL SAFETY CORP,1102 QUEENS PLZ S,LONG ISLAND CITY,NY,11101.0,...,999 Other,2362 Nonresidential Building Construction,2362.0,NEW YORK DISTRICT OFFICE,e $5-10 million,100 to 249,"1102 QUEENS PLZ S, LONG ISLAND CITY, NY",40.752814,-73.946568,NaN
3,605844,1060377105,4/9/20,202,PPP,"CHOP'T HOLDINGS, LLC","853 Broadway, Suite 606 0.0",New York,NY,10003.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"853 Broadway, Suite 606 0.0, New York, NY",40.734690,-73.990790,3.606101e+14
4,605845,1100587104,4/9/20,202,PPP,SENTRY CENTERS HOLDINGS LLC,"101 Greenwich Street, 4th Floor 0.0",New York,NY,10006.0,...,999 Other,5191 Other Information Services,5191.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"101 Greenwich Street, 4th Floor 0.0, New York, NY",40.708027,-74.013560,3.606100e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74108,679949,9808878607,3/26/21,202,PPP,ALT BUSINESS PLATFORMS INC,13159 224th St,Laurelton,NY,11413.0,...,999 Other,2361 Residential Building Construction,2361.0,NEW YORK DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"13159 224th St, Laurelton, NY",40.679641,-73.744327,NaN
74109,679950,9818628508,3/12/21,235,PPS,GREEN WELLNESS CENTER & DAY SPA INC,519 Central Ave,Massapequa,NY,11758.0,...,999 Other,8121 Personal Care Services,8121.0,MELVILLE BRANCH OFFICE,"a $150,000-350,000",20 to 49,"519 Central Ave, Massapequa, NY",40.678092,-73.471767,NaN
74110,679951,9905798609,3/26/21,296,PPS,COMMUNITY ACTION OF ORLEANS AND GENESEE COUNTY...,409 E State St,Albion,NY,14411.0,...,999 Other,"8139 Business, Professional, Labor, Political,...",8139.0,BUFFALO DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"409 E State St, Albion, NY",43.246697,-78.183473,NaN
74111,679952,9985928403,2/18/21,235,PPS,D & D BILLING SERVICS INC,1175 W Broadway,Hewlett,NY,11557.0,...,999 Other,"5419 Other Professional, Scientific, and Techn...",5419.0,MELVILLE BRANCH OFFICE,"a $150,000-350,000",10 to 19,"1175 W Broadway, Hewlett, NY",40.637690,-73.705559,NaN


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long
55613,7963377103,"1 S. Washington Street, Ste 400, NY",NaN,NaN


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,1009208907,"641 Old Route 17, Monticello, NY",41.701466,-74.735090
1,1029567410,"60 EAST 42ND ST SUITE 1250, NEW YORK, NY",40.752097,-73.978733
2,1040387100,"1102 QUEENS PLZ S, LONG ISLAND CITY, NY",40.752814,-73.946568
3,1060377105,"853 Broadway, Suite 606 0.0, New York, NY",40.734690,-73.990790
4,1100587104,"101 Greenwich Street, 4th Floor 0.0, New York, NY",40.708027,-74.013560


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/NY_fips_scraped.csv") 
fips_check_scraped.head() 

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,605841,1009208907,4/24/21,202,PPP,THE CENTER FOR DISCOVERY INC.,641 Old Route 17,...,999 Other,6232 Residential Intellectual and Developmenta...,6232.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"641 Old Route 17, Monticello, NY",41.701466,-74.735090,3.610595e+14
1,1,1,1,605842,1029567410,5/3/20,202,PPP,"BEDDING ACQUISITION, LLC",60 EAST 42ND ST SUITE 1250,...,314 Textile Products,3149 Other Textile Product Mills,3149.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"60 EAST 42ND ST SUITE 1250, NEW YORK, NY",40.752097,-73.978733,3.606101e+14
2,2,2,2,605843,1040387100,4/9/20,202,PPP,P A L ENVIRONMENTAL SAFETY CORP,1102 QUEENS PLZ S,...,999 Other,2362 Nonresidential Building Construction,2362.0,NEW YORK DISTRICT OFFICE,e $5-10 million,100 to 249,"1102 QUEENS PLZ S, LONG ISLAND CITY, NY",40.752814,-73.946568,3.608100e+14
3,3,3,3,605844,1060377105,4/9/20,202,PPP,"CHOP'T HOLDINGS, LLC","853 Broadway, Suite 606 0.0",...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"853 Broadway, Suite 606 0.0, New York, NY",40.734690,-73.990790,3.606101e+14
4,4,4,4,605845,1100587104,4/9/20,202,PPP,SENTRY CENTERS HOLDINGS LLC,"101 Greenwich Street, 4th Floor 0.0",...,999 Other,5191 Other Information Services,5191.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"101 Greenwich Street, 4th Floor 0.0, New York, NY",40.708027,-74.013560,3.606100e+14


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [4]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [5]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

,LoanNumber,full_add,Lat,Long,FIPS_z
0,1009208907,"641 Old Route 17, Monticello, NY",41.701466,-74.735090,3.610595e+14
1,1029567410,"60 EAST 42ND ST SUITE 1250, NEW YORK, NY",40.752097,-73.978733,3.606101e+14
2,1040387100,"1102 QUEENS PLZ S, LONG ISLAND CITY, NY",40.752814,-73.946568,3.608100e+14
3,1060377105,"853 Broadway, Suite 606 0.0, New York, NY",40.734690,-73.990790,3.606101e+14
4,1100587104,"101 Greenwich Street, 4th Floor 0.0, New York, NY",40.708027,-74.013560,3.606100e+14
...,...,...,...,...,...
74108,9808878607,"13159 224th St, Laurelton, NY",40.679641,-73.744327,NaN
74109,9818628508,"519 Central Ave, Massapequa, NY",40.678092,-73.471767,NaN
74110,9905798609,"409 E State St, Albion, NY",43.246697,-78.183473,NaN
74111,9985928403,"1175 W Broadway, Hewlett, NY",40.637690,-73.705559,NaN


In [6]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #5503

,LoanNumber,full_add,Lat,Long,FIPS_z
287,3951297402,"221 E 71st Street 0.0, New York, NY",40.794403,-74.005440,NaN
423,8426367110,"150 E. 58th Street 39th Floor, NEW YORK, NY",40.784798,-74.011215,NaN
587,3057417102,"156 W 56th St 0.0, New York, NY",40.783627,-74.012220,NaN
3809,3060367101,"150 E. 58th St. 39th Floor, NEW YORK, NY",40.784798,-74.011215,NaN
4304,5472187210,"250 W 55TH ST, 35TH FL, NEW YORK, NY",40.784020,-74.014680,NaN
...,...,...,...,...,...
74108,9808878607,"13159 224th St, Laurelton, NY",40.679641,-73.744327,NaN
74109,9818628508,"519 Central Ave, Massapequa, NY",40.678092,-73.471767,NaN
74110,9905798609,"409 E State St, Albion, NY",43.246697,-78.183473,NaN
74111,9985928403,"1175 W Broadway, Hewlett, NY",40.637690,-73.705559,NaN


<h4>FIPS Script

In [7]:
#if geocode file
#lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
#long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [ ]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
service = {'service': Service(ChromeDriverManager().install())}
driver = Browser('chrome', **service, headless=True)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('36')] #NY
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


  0%|          | 0/37785 [00:00<?, ?it/s]

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [9]:
geo_id_list =geo_id 

Length Check

In [10]:
print(len(geo_id_list))

22103


Matching FIPS to DataFrame

In [11]:
#for geocode file
#fips_fill = fips_check_notna.iloc[:10118] #confirm pair to notna DataFrame
#if FIPS available
fips_fill = fips_check_scraped_null.iloc[:22103]

In [12]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/t9/c5y08h4d40j0xvl66n01j2680000gn/T/ipykernel_10344/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,full_add,Lat,Long,FIPS_z
287,3951297402,"221 E 71st Street 0.0, New York, NY",40.794403,-74.005440,[]
423,8426367110,"150 E. 58th Street 39th Floor, NEW YORK, NY",40.784798,-74.011215,[]
587,3057417102,"156 W 56th St 0.0, New York, NY",40.783627,-74.012220,[]
3809,3060367101,"150 E. 58th St. 39th Floor, NEW YORK, NY",40.784798,-74.011215,[]
4304,5472187210,"250 W 55TH ST, 35TH FL, NEW YORK, NY",40.784020,-74.014680,[]
...,...,...,...,...,...
35988,6918978403,"6316 Northern Blvd, East Norwich, NY",40.841985,-73.533991,[360595181001013]
35989,1754357202,"4947 COMMERCIAL DR, YORKVILLE, NY",43.102820,-75.299626,[360650234003018]
35990,3242157305,"16 West 22nd ST Suite 300, NEW YORK, NY",40.741185,-73.991313,[360610058003002]
35991,1412298607,"3422 Steinway St, Long Is City, NY",40.756182,-73.921584,[360810157001002]


Remove brackets

In [13]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/t9/c5y08h4d40j0xvl66n01j2680000gn/T/ipykernel_10344/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,full_add,Lat,Long,FIPS_z
287,3951297402,"221 E 71st Street 0.0, New York, NY",40.794403,-74.005440,NaN
423,8426367110,"150 E. 58th Street 39th Floor, NEW YORK, NY",40.784798,-74.011215,NaN
587,3057417102,"156 W 56th St 0.0, New York, NY",40.783627,-74.012220,NaN
3809,3060367101,"150 E. 58th St. 39th Floor, NEW YORK, NY",40.784798,-74.011215,NaN
4304,5472187210,"250 W 55TH ST, 35TH FL, NEW YORK, NY",40.784020,-74.014680,NaN
...,...,...,...,...,...
35988,6918978403,"6316 Northern Blvd, East Norwich, NY",40.841985,-73.533991,360595181001013
35989,1754357202,"4947 COMMERCIAL DR, YORKVILLE, NY",43.102820,-75.299626,360650234003018
35990,3242157305,"16 West 22nd ST Suite 300, NEW YORK, NY",40.741185,-73.991313,360610058003002
35991,1412298607,"3422 Steinway St, Long Is City, NY",40.756182,-73.921584,360810157001002


In [14]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
287,3951297402,"221 E 71st Street 0.0, New York, NY",40.794403,-74.005440,NaN
423,8426367110,"150 E. 58th Street 39th Floor, NEW YORK, NY",40.784798,-74.011215,NaN
587,3057417102,"156 W 56th St 0.0, New York, NY",40.783627,-74.012220,NaN
3809,3060367101,"150 E. 58th St. 39th Floor, NEW YORK, NY",40.784798,-74.011215,NaN
4304,5472187210,"250 W 55TH ST, 35TH FL, NEW YORK, NY",40.784020,-74.014680,NaN
4936,7599308506,"535 E 70th St Rm 853 Attn Matt Rosenberg, New ...",40.796830,-74.012276,NaN
5526,6927327206,"156 W 56TH ST, 6TH FLOOR, NEW YORK, NY",40.783627,-74.012220,NaN
5729,9898958610,"226 E.54th St - 2nd Fl, New York, NY",40.783060,-74.014400,NaN
6741,9350617110,"145 E 57TH ST 4TH FLOOR, NEW YORK, NY",40.784170,-74.011640,NaN
7054,8937908409,"200 W 70th St Ground Floor, New York, NY",40.793680,-74.005600,NaN


Parse for Merge

In [15]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
287,3951297402,NaN
423,8426367110,NaN
587,3057417102,NaN
3809,3060367101,NaN
4304,5472187210,NaN
...,...,...
35988,6918978403,360595181001013
35989,1754357202,360650234003018
35990,3242157305,360610058003002
35991,1412298607,360810157001002


Read in base data

In [16]:
#if from geocode file
#fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_ny.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1", "Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
#fips_check

#if from FIPS file
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/NY_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1"], axis =1) 
fips_check.head() 


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,605841,1009208907,4/24/21,202,PPP,THE CENTER FOR DISCOVERY INC.,641 Old Route 17,Monticello,...,999 Other,6232 Residential Intellectual and Developmenta...,6232.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"641 Old Route 17, Monticello, NY",41.701466,-74.735090,3.610595e+14
1,1,1,605842,1029567410,5/3/20,202,PPP,"BEDDING ACQUISITION, LLC",60 EAST 42ND ST SUITE 1250,NEW YORK,...,314 Textile Products,3149 Other Textile Product Mills,3149.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"60 EAST 42ND ST SUITE 1250, NEW YORK, NY",40.752097,-73.978733,3.606101e+14
2,2,2,605843,1040387100,4/9/20,202,PPP,P A L ENVIRONMENTAL SAFETY CORP,1102 QUEENS PLZ S,LONG ISLAND CITY,...,999 Other,2362 Nonresidential Building Construction,2362.0,NEW YORK DISTRICT OFFICE,e $5-10 million,100 to 249,"1102 QUEENS PLZ S, LONG ISLAND CITY, NY",40.752814,-73.946568,3.608100e+14
3,3,3,605844,1060377105,4/9/20,202,PPP,"CHOP'T HOLDINGS, LLC","853 Broadway, Suite 606 0.0",New York,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"853 Broadway, Suite 606 0.0, New York, NY",40.734690,-73.990790,3.606101e+14
4,4,4,605845,1100587104,4/9/20,202,PPP,SENTRY CENTERS HOLDINGS LLC,"101 Greenwich Street, 4th Floor 0.0",New York,...,999 Other,5191 Other Information Services,5191.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"101 Greenwich Street, 4th Floor 0.0, New York, NY",40.708027,-74.013560,3.606100e+14


Merge with MainFrame

In [17]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,0,0,605841,1009208907,4/24/21,202,PPP,THE CENTER FOR DISCOVERY INC.,641 Old Route 17,Monticello,...,6232 Residential Intellectual and Developmenta...,6232.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"641 Old Route 17, Monticello, NY",41.701466,-74.735090,3.610595e+14,NaN
1,1,1,605842,1029567410,5/3/20,202,PPP,"BEDDING ACQUISITION, LLC",60 EAST 42ND ST SUITE 1250,NEW YORK,...,3149 Other Textile Product Mills,3149.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"60 EAST 42ND ST SUITE 1250, NEW YORK, NY",40.752097,-73.978733,3.606101e+14,NaN
2,2,2,605843,1040387100,4/9/20,202,PPP,P A L ENVIRONMENTAL SAFETY CORP,1102 QUEENS PLZ S,LONG ISLAND CITY,...,2362 Nonresidential Building Construction,2362.0,NEW YORK DISTRICT OFFICE,e $5-10 million,100 to 249,"1102 QUEENS PLZ S, LONG ISLAND CITY, NY",40.752814,-73.946568,3.608100e+14,NaN
3,3,3,605844,1060377105,4/9/20,202,PPP,"CHOP'T HOLDINGS, LLC","853 Broadway, Suite 606 0.0",New York,...,7225 Restaurants and Other Eating Places,7225.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"853 Broadway, Suite 606 0.0, New York, NY",40.734690,-73.990790,3.606101e+14,NaN
4,4,4,605845,1100587104,4/9/20,202,PPP,SENTRY CENTERS HOLDINGS LLC,"101 Greenwich Street, 4th Floor 0.0",New York,...,5191 Other Information Services,5191.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"101 Greenwich Street, 4th Floor 0.0, New York, NY",40.708027,-74.013560,3.606100e+14,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74108,74108,74108,679949,9808878607,3/26/21,202,PPP,ALT BUSINESS PLATFORMS INC,13159 224th St,Laurelton,...,2361 Residential Building Construction,2361.0,NEW YORK DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"13159 224th St, Laurelton, NY",40.679641,-73.744327,NaN,NaN
74109,74109,74109,679950,9818628508,3/12/21,235,PPS,GREEN WELLNESS CENTER & DAY SPA INC,519 Central Ave,Massapequa,...,8121 Personal Care Services,8121.0,MELVILLE BRANCH OFFICE,"a $150,000-350,000",20 to 49,"519 Central Ave, Massapequa, NY",40.678092,-73.471767,NaN,NaN
74110,74110,74110,679951,9905798609,3/26/21,296,PPS,COMMUNITY ACTION OF ORLEANS AND GENESEE COUNTY...,409 E State St,Albion,...,"8139 Business, Professional, Labor, Political,...",8139.0,BUFFALO DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"409 E State St, Albion, NY",43.246697,-78.183473,NaN,NaN
74111,74111,74111,679952,9985928403,2/18/21,235,PPS,D & D BILLING SERVICS INC,1175 W Broadway,Hewlett,...,"5419 Other Professional, Scientific, and Techn...",5419.0,MELVILLE BRANCH OFFICE,"a $150,000-350,000",10 to 19,"1175 W Broadway, Hewlett, NY",40.637690,-73.705559,NaN,NaN


<b>Fill FIPS Columns - if FIPS file

In [18]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,0,0,605841,1009208907,4/24/21,202,PPP,THE CENTER FOR DISCOVERY INC.,641 Old Route 17,Monticello,...,6232.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"641 Old Route 17, Monticello, NY",41.701466,-74.735090,3.610595e+14,NaN,361059515002013.0
1,1,1,605842,1029567410,5/3/20,202,PPP,"BEDDING ACQUISITION, LLC",60 EAST 42ND ST SUITE 1250,NEW YORK,...,3149.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"60 EAST 42ND ST SUITE 1250, NEW YORK, NY",40.752097,-73.978733,3.606101e+14,NaN,360610082002000.0
2,2,2,605843,1040387100,4/9/20,202,PPP,P A L ENVIRONMENTAL SAFETY CORP,1102 QUEENS PLZ S,LONG ISLAND CITY,...,2362.0,NEW YORK DISTRICT OFFICE,e $5-10 million,100 to 249,"1102 QUEENS PLZ S, LONG ISLAND CITY, NY",40.752814,-73.946568,3.608100e+14,NaN,360810019031008.0
3,3,3,605844,1060377105,4/9/20,202,PPP,"CHOP'T HOLDINGS, LLC","853 Broadway, Suite 606 0.0",New York,...,7225.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"853 Broadway, Suite 606 0.0, New York, NY",40.734690,-73.990790,3.606101e+14,NaN,360610061003001.0
4,4,4,605845,1100587104,4/9/20,202,PPP,SENTRY CENTERS HOLDINGS LLC,"101 Greenwich Street, 4th Floor 0.0",New York,...,5191.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"101 Greenwich Street, 4th Floor 0.0, New York, NY",40.708027,-74.013560,3.606100e+14,NaN,360610013003001.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74108,74108,74108,679949,9808878607,3/26/21,202,PPP,ALT BUSINESS PLATFORMS INC,13159 224th St,Laurelton,...,2361.0,NEW YORK DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"13159 224th St, Laurelton, NY",40.679641,-73.744327,NaN,NaN,NaN
74109,74109,74109,679950,9818628508,3/12/21,235,PPS,GREEN WELLNESS CENTER & DAY SPA INC,519 Central Ave,Massapequa,...,8121.0,MELVILLE BRANCH OFFICE,"a $150,000-350,000",20 to 49,"519 Central Ave, Massapequa, NY",40.678092,-73.471767,NaN,NaN,NaN
74110,74110,74110,679951,9905798609,3/26/21,296,PPS,COMMUNITY ACTION OF ORLEANS AND GENESEE COUNTY...,409 E State St,Albion,...,8139.0,BUFFALO DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"409 E State St, Albion, NY",43.246697,-78.183473,NaN,NaN,NaN
74111,74111,74111,679952,9985928403,2/18/21,235,PPS,D & D BILLING SERVICS INC,1175 W Broadway,Hewlett,...,5419.0,MELVILLE BRANCH OFFICE,"a $150,000-350,000",10 to 19,"1175 W Broadway, Hewlett, NY",40.637690,-73.705559,NaN,NaN,NaN


Drop Extra FIPS Columns

In [19]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,605841,1009208907,4/24/21,202,PPP,THE CENTER FOR DISCOVERY INC.,641 Old Route 17,Monticello,...,999 Other,6232 Residential Intellectual and Developmenta...,6232.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"641 Old Route 17, Monticello, NY",41.701466,-74.735090,361059515002013.0
1,1,1,605842,1029567410,5/3/20,202,PPP,"BEDDING ACQUISITION, LLC",60 EAST 42ND ST SUITE 1250,NEW YORK,...,314 Textile Products,3149 Other Textile Product Mills,3149.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"60 EAST 42ND ST SUITE 1250, NEW YORK, NY",40.752097,-73.978733,360610082002000.0
2,2,2,605843,1040387100,4/9/20,202,PPP,P A L ENVIRONMENTAL SAFETY CORP,1102 QUEENS PLZ S,LONG ISLAND CITY,...,999 Other,2362 Nonresidential Building Construction,2362.0,NEW YORK DISTRICT OFFICE,e $5-10 million,100 to 249,"1102 QUEENS PLZ S, LONG ISLAND CITY, NY",40.752814,-73.946568,360810019031008.0
3,3,3,605844,1060377105,4/9/20,202,PPP,"CHOP'T HOLDINGS, LLC","853 Broadway, Suite 606 0.0",New York,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"853 Broadway, Suite 606 0.0, New York, NY",40.734690,-73.990790,360610061003001.0
4,4,4,605845,1100587104,4/9/20,202,PPP,SENTRY CENTERS HOLDINGS LLC,"101 Greenwich Street, 4th Floor 0.0",New York,...,999 Other,5191 Other Information Services,5191.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"101 Greenwich Street, 4th Floor 0.0, New York, NY",40.708027,-74.013560,360610013003001.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74108,74108,74108,679949,9808878607,3/26/21,202,PPP,ALT BUSINESS PLATFORMS INC,13159 224th St,Laurelton,...,999 Other,2361 Residential Building Construction,2361.0,NEW YORK DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"13159 224th St, Laurelton, NY",40.679641,-73.744327,NaN
74109,74109,74109,679950,9818628508,3/12/21,235,PPS,GREEN WELLNESS CENTER & DAY SPA INC,519 Central Ave,Massapequa,...,999 Other,8121 Personal Care Services,8121.0,MELVILLE BRANCH OFFICE,"a $150,000-350,000",20 to 49,"519 Central Ave, Massapequa, NY",40.678092,-73.471767,NaN
74110,74110,74110,679951,9905798609,3/26/21,296,PPS,COMMUNITY ACTION OF ORLEANS AND GENESEE COUNTY...,409 E State St,Albion,...,999 Other,"8139 Business, Professional, Labor, Political,...",8139.0,BUFFALO DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"409 E State St, Albion, NY",43.246697,-78.183473,NaN
74111,74111,74111,679952,9985928403,2/18/21,235,PPS,D & D BILLING SERVICS INC,1175 W Broadway,Hewlett,...,999 Other,"5419 Other Professional, Scientific, and Techn...",5419.0,MELVILLE BRANCH OFFICE,"a $150,000-350,000",10 to 19,"1175 W Broadway, Hewlett, NY",40.637690,-73.705559,NaN


In [20]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
287,287,287,606128,3951297402,5/8/20,202,PPP,MARYMOUNT MANHATTAN COLLEGE,221 E 71st Street 0.0,New York,...,999 Other,"6113 Colleges, Universities, and Professional ...",6113.0,NEW YORK DISTRICT OFFICE,e $5-10 million,250 to 499,"221 E 71st Street 0.0, New York, NY",40.794403,-74.005440,NaN
423,423,423,606264,8426367110,4/15/20,202,PPP,ICONIC COPLEY PLAZA HOTEL LLC,150 E. 58th Street 39th Floor,NEW YORK,...,999 Other,7211 Traveler Accommodation,7211.0,NEW YORK DISTRICT OFFICE,e $5-10 million,250 to 499,"150 E. 58th Street 39th Floor, NEW YORK, NY",40.784798,-74.011215,NaN
587,587,587,606428,3057417102,4/11/20,202,PPP,"NEW YORK ROAD RUNNERS, INC.",156 W 56th St 0.0,New York,...,999 Other,7112 Spectator Sports,7112.0,NEW YORK DISTRICT OFFICE,d $2-5 million,250 to 499,"156 W 56th St 0.0, New York, NY",40.783627,-74.012220,NaN
3809,3809,3809,609650,3060367101,4/11/20,202,PPP,NY BROADWAY HOTEL OWNER LLC,150 E. 58th St. 39th Floor,NEW YORK,...,999 Other,7211 Traveler Accommodation,7211.0,NEW YORK DISTRICT OFFICE,c $1-2 million,50 to 99,"150 E. 58th St. 39th Floor, NEW YORK, NY",40.784798,-74.011215,NaN
4304,4304,4304,610145,5472187210,4/27/20,202,PPP,"FAIRSTEAD MANAGEMENT, LLC","250 W 55TH ST, 35TH FL",NEW YORK,...,999 Other,5313 Activities Related to Real Estate,5313.0,NEW YORK DISTRICT OFFICE,c $1-2 million,100 to 249,"250 W 55TH ST, 35TH FL, NEW YORK, NY",40.784020,-74.014680,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74108,74108,74108,679949,9808878607,3/26/21,202,PPP,ALT BUSINESS PLATFORMS INC,13159 224th St,Laurelton,...,999 Other,2361 Residential Building Construction,2361.0,NEW YORK DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"13159 224th St, Laurelton, NY",40.679641,-73.744327,NaN
74109,74109,74109,679950,9818628508,3/12/21,235,PPS,GREEN WELLNESS CENTER & DAY SPA INC,519 Central Ave,Massapequa,...,999 Other,8121 Personal Care Services,8121.0,MELVILLE BRANCH OFFICE,"a $150,000-350,000",20 to 49,"519 Central Ave, Massapequa, NY",40.678092,-73.471767,NaN
74110,74110,74110,679951,9905798609,3/26/21,296,PPS,COMMUNITY ACTION OF ORLEANS AND GENESEE COUNTY...,409 E State St,Albion,...,999 Other,"8139 Business, Professional, Labor, Political,...",8139.0,BUFFALO DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"409 E State St, Albion, NY",43.246697,-78.183473,NaN
74111,74111,74111,679952,9985928403,2/18/21,235,PPS,D & D BILLING SERVICS INC,1175 W Broadway,Hewlett,...,999 Other,"5419 Other Professional, Scientific, and Techn...",5419.0,MELVILLE BRANCH OFFICE,"a $150,000-350,000",10 to 19,"1175 W Broadway, Hewlett, NY",40.637690,-73.705559,NaN


Convert to File

In [21]:
fips_NY = fips_merge_drop
#fips_TN = fips_merge_drop

In [22]:
fips_NY.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/NY_fips_scraped.csv") 

Review & Compare

In [23]:
fips_NY = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/NY_fips_scraped.csv") 
fips_NY

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,605841,1009208907,4/24/21,202,PPP,THE CENTER FOR DISCOVERY INC.,641 Old Route 17,...,999 Other,6232 Residential Intellectual and Developmenta...,6232.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"641 Old Route 17, Monticello, NY",41.701466,-74.735090,3.610595e+14
1,1,1,1,605842,1029567410,5/3/20,202,PPP,"BEDDING ACQUISITION, LLC",60 EAST 42ND ST SUITE 1250,...,314 Textile Products,3149 Other Textile Product Mills,3149.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"60 EAST 42ND ST SUITE 1250, NEW YORK, NY",40.752097,-73.978733,3.606101e+14
2,2,2,2,605843,1040387100,4/9/20,202,PPP,P A L ENVIRONMENTAL SAFETY CORP,1102 QUEENS PLZ S,...,999 Other,2362 Nonresidential Building Construction,2362.0,NEW YORK DISTRICT OFFICE,e $5-10 million,100 to 249,"1102 QUEENS PLZ S, LONG ISLAND CITY, NY",40.752814,-73.946568,3.608100e+14
3,3,3,3,605844,1060377105,4/9/20,202,PPP,"CHOP'T HOLDINGS, LLC","853 Broadway, Suite 606 0.0",...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"853 Broadway, Suite 606 0.0, New York, NY",40.734690,-73.990790,3.606101e+14
4,4,4,4,605845,1100587104,4/9/20,202,PPP,SENTRY CENTERS HOLDINGS LLC,"101 Greenwich Street, 4th Floor 0.0",...,999 Other,5191 Other Information Services,5191.0,NEW YORK DISTRICT OFFICE,e $5-10 million,500 or more,"101 Greenwich Street, 4th Floor 0.0, New York, NY",40.708027,-74.013560,3.606100e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74108,74108,74108,74108,679949,9808878607,3/26/21,202,PPP,ALT BUSINESS PLATFORMS INC,13159 224th St,...,999 Other,2361 Residential Building Construction,2361.0,NEW YORK DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"13159 224th St, Laurelton, NY",40.679641,-73.744327,NaN
74109,74109,74109,74109,679950,9818628508,3/12/21,235,PPS,GREEN WELLNESS CENTER & DAY SPA INC,519 Central Ave,...,999 Other,8121 Personal Care Services,8121.0,MELVILLE BRANCH OFFICE,"a $150,000-350,000",20 to 49,"519 Central Ave, Massapequa, NY",40.678092,-73.471767,NaN
74110,74110,74110,74110,679951,9905798609,3/26/21,296,PPS,COMMUNITY ACTION OF ORLEANS AND GENESEE COUNTY...,409 E State St,...,999 Other,"8139 Business, Professional, Labor, Political,...",8139.0,BUFFALO DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"409 E State St, Albion, NY",43.246697,-78.183473,NaN
74111,74111,74111,74111,679952,9985928403,2/18/21,235,PPS,D & D BILLING SERVICS INC,1175 W Broadway,...,999 Other,"5419 Other Professional, Scientific, and Techn...",5419.0,MELVILLE BRANCH OFFICE,"a $150,000-350,000",10 to 19,"1175 W Broadway, Hewlett, NY",40.637690,-73.705559,NaN
